In [189]:
#FIFA Rank
#Source:https://www.kaggle.com/datasets/cashncarry/fifaworldranking?resource=download
from sklearnex import patch_sklearn                         #intel 加速補丁
patch_sklearn()                                             #这个补丁之后的所有import语句导入的来自sklearn的算法，都会优先导入Scikit-Learn优化版本
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import time
import math
import re
pd.set_option('display.max_columns', 500)

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [191]:
def ability_search(player_list=["Manuel Peter Neuer","Adam Kelsey","Leslie Sackey"],year=2018):
    #print("Search")
    if year==2018:
        df=pd.read_csv("ability_18.csv",encoding = "utf-8")
    elif year==2022:
        df=pd.read_csv("ability_22.csv",encoding = "utf-8")

    sum=pd.DataFrame()
    column_means=pd.DataFrame()
    for name in player_list:
        name=name.replace('-',' ')
        if df["long_name"].str.contains(name).any():
            form=df[df["long_name"]==name].iloc[:,2:]
            sum=pd.concat([sum,form],axis=0,ignore_index=True)
            #display(sum)
        #else:
        #   print(name," :Not found")
    sum=sum.fillna(0)
    sum = sum.apply(pd.to_numeric, errors='coerce')
    column_means = sum[sum != 0].mean().to_frame().transpose()
    #display(column_means)
    #print("Down")           
    return column_means

'''
test=pd.read_csv("2022 World Cup Player.csv",encoding="UTF-8")
test=test.iloc[:,1:]
#print(test.iloc[0,:].to_list())
for i in range(1,32):
    ability_search(player_list=test.iloc[i,:].to_list())
#ability_search()
'''

'\ntest=pd.read_csv("2022 World Cup Player.csv",encoding="UTF-8")\ntest=test.iloc[:,1:]\n#print(test.iloc[0,:].to_list())\nfor i in range(1,32):\n    ability_search(player_list=test.iloc[i,:].to_list())\n#ability_search()\n'

In [192]:
#input 為對戰表
def info_search(home="Ecuador",away="Netherlands",date="2018-06-14"):
    
    df=pd.read_csv("fifa_ranking-2022-12-22.csv",encoding="UTF-8")
    player=pd.read_csv("2018 World Cup Player.csv",encoding="utf-8-sig",thousands=',')         #記得修改dataset

    df=df[df["rank_date"]<date]
    home_rank=0
    away_rank=0
    ability_diff=pd.DataFrame()
    #home_ability=pd.DataFrame()
    #away_ability=pd.DataFrame()
    # Use nation to find range
    if df["country_full"].str.contains(home).any():
        home_rank=(df[df["country_full"]==home]["rank"].iloc[-1])
    else:
        print(home," :Home Team Not Found")
    if df["country_full"].str.contains(away).any():
        #print("Away Team: ",away,df[df["country_full"]==away])
        away_rank=(df[df["country_full"]==away]["rank"].iloc[-1])
    else:
        print(away," : Away Team Not Found")

    # Use nation to get player list
     #home team
    if player["Nation"].str.contains(home).any():
        index=player[player["Nation"]==home].index
        #print("home team: ",home," index: ",index)
        players=player.iloc[index,1:].dropna(axis=1).values.tolist()[0]
        #print(players)
        players=[x.lower() for x in players]
        home_ability=ability_search(player_list=players,year=2022)
        #display(ability_search(player_list=players,year=2022))
        #print("Team: ",home,home_ability,'\n')
    else:
        print(home," :Home Team Not Found in word_cup_simulation.csv")    
     #away team 
    if player["Nation"].str.contains(away).any():
        index=player[player["Nation"]==away].index
        #print("away team: ",away," index: ",index)
        players=player.iloc[index,1:].dropna(axis=1).values.tolist()[0]
        #print(players)
        players=[x.lower() for x in players]
        away_ability=ability_search(player_list=players,year=2022)
        #print("Team: ",away,away_ability,'\n')
    else:
        print(away," :away Team Not Found in word_cup_simulation.csv")
    #print('home: ',home)
    #display(home_ability)
    #print('away: ',away)
    #display(away_ability)
    ability_diff=home_ability-away_ability
 
    rank_diff=pd.DataFrame({"rank_diff":[(home_rank-away_rank)]})
    #display("Diff: ",ability_diff)
    #display("Rank diff: ",rank_diff)
    ability_diff=pd.concat((ability_diff,rank_diff),axis=1,ignore_index=False)
    #display(ability_diff)
    return ability_diff

#display(info_search())


In [193]:
game22=pd.read_csv("2018 World Cup Game.csv",encoding = "utf-8-sig") 
display(game22)

,year,Group(小組分組),Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Goal_dif,Result
0,2018,Group A,Russia,5,0,Saudi Arabia,5,1
1,2018,Group A,Egypt,0,1,Uruguay,-1,0
2,2018,Group B,Portugal,3,3,Spain,0,0
3,2018,Group B,Morocco,0,1,IR Iran,-1,0
4,2018,Group C,France,2,1,Australia,1,1
...,...,...,...,...,...,...,...,...
59,2018,Quarter-final,Sweden,0,2,England,-2,0
60,2018,Semi-final,France,1,0,Belgium,1,1
61,2018,Semi-final,Croatia,2,1,England,1,1
62,2018,Play-off for third place,Belgium,2,0,England,2,1


In [194]:
#total ability info
info=pd.DataFrame()
for i in range(0,64,1):
    print("Game ",i)
    home=game22["Home Team Name"].iloc[i]
    away=game22["Away Team Name"].iloc[i]
    print("home: ",home," Away: ",away)
    frame=info_search(home=home,away=away)
    #display(frame)
    info=pd.concat((info,frame),axis=0)
    #display(info)
print("total diff: ")
display(info)

    

Game  0
home:  Russia  Away:  Saudi Arabia
Game  1
home:  Egypt  Away:  Uruguay
Game  2
home:  Portugal  Away:  Spain
Game  3
home:  Morocco  Away:  IR Iran
Game  4
home:  France  Away:  Australia
Game  5
home:  Peru  Away:  Denmark
Game  6
home:  Argentina  Away:  Iceland
Game  7
home:  Croatia  Away:  Nigeria
Game  8
home:  Brazil  Away:  Switzerland
Game  9
home:  Costa Rica  Away:  Serbia
Game  10
home:  Germany  Away:  Mexico
Game  11
home:  Sweden  Away:  Korea Republic
Game  12
home:  Belgium  Away:  Panama
Game  13
home:  Tunisia  Away:  England
Game  14
home:  Poland  Away:  Senegal
Game  15
home:  Colombia  Away:  Japan
Game  16
home:  Russia  Away:  Egypt
Game  17
home:  Uruguay  Away:  Saudi Arabia
Game  18
home:  Portugal  Away:  Morocco
Game  19
home:  IR Iran  Away:  Spain
Game  20
home:  France  Away:  Peru
Game  21
home:  Denmark  Away:  Australia
Game  22
home:  Argentina  Away:  France
Game  23
home:  Nigeria  Away:  Iceland
Game  24
home:  Brazil  Away:  Costa Rica


,overall,age,height_cm,weight_kg,skill_moves,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,rank_diff
0,7.337662,0.207792,6.922078,7.766234,0.311688,-5.444444,12.277778,9.111111,5.388889,-4.277778,1.777778,8.285714,14.064935,8.181818,8.688312,10.259740,7.480519,7.428571,15.974026,9.935065,9.818182,-2.974026,-1.402597,-4.454545,9.675325,-3.662338,11.636364,-8.181818,4.415584,5.233766,9.376623,-0.558442,-1.311688,10.506494,10.805195,8.519481,9.701299,-5.389610,-2.896104,-0.857143,0.285714,-3.935065,2.025974,1.064935,-0.051948,10.000000,1
0,-1.115385,-1.846154,-2.807692,1.153846,0.230769,-3.727273,-10.363636,-4.409091,-2.181818,-1.750000,0.272727,-2.346154,-2.711538,-1.923077,3.365385,-5.865385,5.403846,-0.730769,-1.211538,7.230769,5.326923,-1.384615,1.000000,3.557692,-0.865385,5.807692,-8.461538,1.307692,8.923077,3.750000,0.903846,3.788462,9.346154,-2.576923,-3.653846,-3.653846,-0.673077,5.942308,4.807692,7.500000,-8.826923,-5.807692,-13.596154,-12.365385,-12.750000,NaN,10
0,4.888889,3.444444,-4.666667,-12.000000,2.444444,NaN,NaN,NaN,NaN,NaN,NaN,43.000000,56.222222,43.777778,25.888889,44.333333,63.666667,44.555556,40.000000,20.000000,42.000000,38.777778,37.222222,34.888889,15.333333,32.111111,12.000000,2.666667,40.555556,20.111111,54.666667,50.333333,39.333333,53.555556,17.000000,40.444444,20.555556,35.888889,40.777778,35.444444,-60.444444,-58.333333,-68.666667,-57.444444,-62.333333,29.000000,-3
0,3.979167,1.930556,-1.638889,-2.847222,0.458333,1.361905,2.123810,4.180952,3.771429,3.209524,1.466667,12.555556,12.125000,6.006944,12.013889,10.430556,14.333333,13.583333,8.875000,9.555556,11.354167,6.763889,6.243056,7.604167,3.958333,6.187500,3.354167,-0.909722,9.722222,0.090278,9.659722,13.659722,10.048611,12.812500,8.861111,9.645833,9.131944,8.916667,9.423611,9.145833,-8.444444,-8.381944,-7.381944,-9.770833,-10.444444,1.000000,8
0,11.551487,-1.434783,1.906178,0.448513,0.247140,2.325000,4.862500,10.537500,9.937500,8.900000,2.962500,7.421053,3.318078,10.441648,13.745995,9.551487,10.050343,7.167048,10.945080,13.167048,13.533181,1.974828,3.524027,5.457666,14.526316,0.151030,8.610984,5.000000,2.590389,0.519451,5.816934,6.185355,9.167048,6.853547,10.649886,3.118993,12.407323,8.803204,8.418764,9.235698,-1.048055,-1.402746,-1.514874,-1.933638,-1.336384,2.666667,-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,-5.850000,2.208333,2.216667,1.516667,-0.133333,-11.882353,-6.439776,-4.630252,-5.467787,-8.002801,-0.890756,-7.125000,-8.325000,-7.766667,-4.575000,-5.291667,-7.083333,-3.750000,-3.183333,-5.783333,-6.541667,-11.525000,-11.433333,-4.966667,-5.258333,-7.225000,-5.275000,-9.750000,-1.466667,0.558333,-6.250000,-7.125000,-7.808333,-9.466667,-4.791667,-7.408333,-4.858333,-6.758333,-9.058333,-9.608333,0.558333,0.233333,1.408333,-0.333333,-0.025000,-6.333333,3
0,0.584980,-1.434783,-2.213439,-2.175889,-0.219368,1.818421,-4.778947,-1.097368,-0.184211,4.636842,3.281579,-1.000000,-4.533597,2.288538,1.501976,-4.551383,-0.693676,-2.679842,-4.466403,1.456522,1.581028,3.051383,2.260870,2.507905,2.318182,-1.171937,-2.879447,6.318182,3.832016,-0.171937,-4.312253,5.960474,5.820158,-2.758893,0.367589,-5.428854,-0.162055,4.616601

In [195]:
display(game22.columns)
print(len(game22.columns) == len(set(game22.columns)))
new=pd.concat([game22,info.reset_index(drop=True)],axis=1)
#new.to_csv("2018_result.csv",encoding="utf-8-sig")
display(new)

Index(['year', 'Group(小組分組)', 'Home Team Name', 'Home Team Goals',
       'Away Team Goals', 'Away Team Name', 'Goal_dif', 'Result'],
      dtype='object')

True


,year,Group(小組分組),Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Goal_dif,Result,overall,age,height_cm,weight_kg,skill_moves,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,rank_diff
0,2018,Group A,Russia,5,0,Saudi Arabia,5,1,7.337662,0.207792,6.922078,7.766234,0.311688,-5.444444,12.277778,9.111111,5.388889,-4.277778,1.777778,8.285714,14.064935,8.181818,8.688312,10.259740,7.480519,7.428571,15.974026,9.935065,9.818182,-2.974026,-1.402597,-4.454545,9.675325,-3.662338,11.636364,-8.181818,4.415584,5.233766,9.376623,-0.558442,-1.311688,10.506494,10.805195,8.519481,9.701299,-5.389610,-2.896104,-0.857143,0.285714,-3.935065,2.025974,1.064935,-0.051948,10.000000,1
1,2018,Group A,Egypt,0,1,Uruguay,-1,0,-1.115385,-1.846154,-2.807692,1.153846,0.230769,-3.727273,-10.363636,-4.409091,-2.181818,-1.750000,0.272727,-2.346154,-2.711538,-1.923077,3.365385,-5.865385,5.403846,-0.730769,-1.211538,7.230769,5.326923,-1.384615,1.000000,3.557692,-0.865385,5.807692,-8.461538,1.307692,8.923077,3.750000,0.903846,3.788462,9.346154,-2.576923,-3.653846,-3.653846,-0.673077,5.942308,4.807692,7.500000,-8.826923,-5.807692,-13.596154,-12.365385,-12.750000,NaN,10
2,2018,Group B,Portugal,3,3,Spain,0,0,4.888889,3.444444,-4.666667,-12.000000,2.444444,NaN,NaN,NaN,NaN,NaN,NaN,43.000000,56.222222,43.777778,25.888889,44.333333,63.666667,44.555556,40.000000,20.000000,42.000000,38.777778,37.222222,34.888889,15.333333,32.111111,12.000000,2.666667,40.555556,20.111111,54.666667,50.333333,39.333333,53.555556,17.000000,40.444444,20.555556,35.888889,40.777778,35.444444,-60.444444,-58.333333,-68.666667,-57.444444,-62.333333,29.000000,-3
3,2018,Group B,Morocco,0,1,IR Iran,-1,0,3.979167,1.930556,-1.638889,-2.847222,0.458333,1.361905,2.123810,4.180952,3.771429,3.209524,1.466667,12.555556,12.125000,6.006944,12.013889,10.430556,14.333333,13.583333,8.875000,9.555556,11.354167,6.763889,6.243056,7.604167,3.958333,6.187500,3.354167,-0.909722,9.722222,0.090278,9.659722,13.659722,10.048611,12.812500,8.861111,9.645833,9.131944,8.916667,9.423611,9.145833,-8.444444,-8.381944,-7.381944,-9.770833,-10.444444,1.000000,8
4,2018,Group C,France,2,1,Australia,1,1,11.551487,-1.434783,1.906178,0.448513,0.247140,2.325000,4.862500,10.537500,9.937500,8.900000,2.962500,7.421053,3.318078,10.441648,13.745995,9.551487,10.050343,7.167048,10.945080,13.167048,13.533181,1.974828,3.524027,5.457666,14.526316,0.151030,8.610984,5.000000,2.590389,0.519451,5.816934,6.185355,9.167048,6.853547,10.649886,3.118993,12.407323,8.803204,8.418764,9.235698,-1.048055,-1.402746,-1.514874,-1.933638,-1.336384,2.666667,-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,2018,Quarter-final,Sweden,0,2,England,-2,0,-5.850000,2.208333,2.216667,1.516667,-0.133333,-11.882353,-6.439776,-4.630252,-5.467787,-8.002801,-0.890756,-7.125000,-8.325000,-7.766667,-4.575000,-5.291667,-7.083333,-3.750000,-3.183333,-5.783333,-6.541667,-11.525000,-11.433333,-4.966667,-5.258333,-7.225000,-5.275000,-9.750000,-1.466667,0.558333,-6.250000,-7.125000,-7.808333,-9.466667,-4.791667,-7.408333,-4.858333,-6.758333,-9.058333,-9.608333,0.558333,0.233333,1.408333,-0.333333,-0.025000,-6.333333,3
60,